In [ ]:
# 득점을 라벨로 두고, 나머지 속성을 엑스로 -> logistic regression
# 전체속성 (득점 추가, 제외 둘다 시도)을 엑스로 -> decision tree, logistic classification, fully connected

In [2]:
import tensorflow as tf
import pandas as pd
from sklearn.cross_validation import train_test_split 
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

/Users/gracelee/.pyenv/versions/3.5.4/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/gracelee/.pyenv/versions/3.5.4/envs/for_jupyter/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [3]:
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

In [5]:
play_win = pd.read_csv('./data/embeded_with_label.tsv', sep='\t')
play_win = play_win.loc[play_win.winning_flag!='draw']
play_win.winning_flag = play_win.winning_flag.apply(lambda x: 0 if x.find('lose')>=0 else 1)
# play_win.winning_flag = 0 if play_win.winning_flag.str.find('lose')>=0 else 1
play_win.describe()

,match_id,player_id,2YR,ASSIST,CA,CC,DAT,DC,DCIP,DCNIP,...,TZONE2,TZONE3,TZONE4,TZONE5,Total_PA,Total_PC,Total_PP,YC,team_id,winning_flag
count,1419.000000,1419.000000,1419.000000,1419.000000,1419.000000,1419.000000,1419.000000,1419.000000,1419.000000,1419.000000,...,1419.000000,1419.000000,1419.000000,1419.000000,1419.000000,1419.000000,1419.000000,1419.000000,1419.000000,1419.000000
mean,32.045807,275.176885,0.001409,0.058492,1.754052,1.391121,2.568710,7486.122622,2721.786469,2927.963354,...,0.236117,0.050282,0.015610,0.005116,31.641297,26.485553,0.784708,0.119098,17.342495,0.499648
std,18.559097,155.749660,0.037529,0.243600,2.363504,1.913490,2.976169,3325.493275,1430.605779,1563.900050,...,0.073101,0.021923,0.008925,0.005637,22.841584,20.965760,0.197187,0.324018,9.182200,0.500176
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,16.000000,139.500000,0.000000,0.000000,0.000000,0.000000,0.000000,4459.500000,1550.500000,1532.500000,...,0.210000,0.040000,0.010000,0.000000,15.000000,11.000000,0.730000,0.000000,9.000000,0.000000
50%,30.000000,287.000000,0.000000,0.000000,1.000000,1.000000,2.000000,8808.000000,2930.000000,3197.000000,...,0.250000,0.050000,0.020000,0.000000,27.000000,21.000000,0.830000,0.000000,18.000000,0.000000
75%,48.000000,384.500000,0.000000,0.000000,3.000000,2.000000,4.000000,9939.500000,3789.000000,4104.000000,...,0.280000,0.060000,0.020000,0.010000,43.000000,37.000000,0.910000,0.000000,25.000000,1.000000
max,64.000000,605.000000,1.000000,2.000000,17.000000,12.000000,26.000000,16339.000000,6553.000000,7000.000000,...,0.420000,0.170000,0.090000,0.060000,127.000000,121.000000,1.000000,1.000000,32.000000,1.000000


In [6]:
play_win.columns

Index(['match_id', 'player_id', '2YR', 'ASSIST', 'CA', 'CC', 'DAT', 'DC',
       'DCIP', 'DCNIP', 'DPA', 'DZONE1', 'DZONE2', 'DZONE3', 'DZONE4',
       'DZONE5', 'FOUL', 'Long_PA', 'Long_PC', 'Medium_PA', 'Medium_PC', 'OFF',
       'POS', 'RC', 'SAVE', 'SCORE', 'SHOT', 'SPA', 'SPRINT', 'SRAT', 'SRPA',
       'SUB', 'Short_PA', 'Short_PC', 'TA3RD', 'TG', 'TNG', 'TOH', 'TPA', 'TS',
       'TSCORE', 'TSCORE_OG', 'TSCORE_PEN', 'TSL', 'TSNL', 'TTP', 'TZONE1',
       'TZONE2', 'TZONE3', 'TZONE4', 'TZONE5', 'Total_PA', 'Total_PC',
       'Total_PP', 'YC', 'team_id', 'winning_flag'],
      dtype='object')

In [7]:
for ind, grp in play_win.groupby('POS'):
    print('==================================')
    print(ind)
    reduced_play_win = grp[['2YR', 'ASSIST', 'CA', 'CC', \
                            'DAT', 'DC', 'DCIP', 'DPA', \
                            'DZONE1', 'DZONE2', 'DZONE3', 'DZONE4', 'DZONE5',\
                            'FOUL', 'Long_PA', 'Long_PC', 'Medium_PA', 'Medium_PC',\
                            'OFF', 'RC', 'SAVE', 'SCORE', 'SHOT', 'SPA', 'SPRINT', 'SRAT',\
                            'SRPA', 'Short_PA', 'Short_PC', 'TA3RD', 'TG', 'TNG', 'TOH', \
                            'TPA', 'TS', 'TSCORE', 'TSCORE_OG', 'TSCORE_PEN', 'TSL', 'TSNL', 'TTP',\
                            'TZONE1', 'TZONE2', 'TZONE3', 'TZONE4', 'TZONE5', 'Total_PA',\
                            'Total_PC', 'Total_PP', 'YC', 'winning_flag']]
    
    print(len(reduced_play_win))
    print(reduced_play_win.shape)
    reduced_play_win.describe()
    reduced_play_win.reset_index(drop=True)
    reduced_play_win = reduced_play_win.values
    X = reduced_play_win[:, :50]
    Y = reduced_play_win[:, 50]
    
    #model
    model = Sequential()
    model.add(Dense(60, input_dim=50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    model.fit(X, Y, epochs=30, batch_size=10)
    print('\n Accuracy: %.4f' % (model.evaluate(X, Y)[1]))

Defender
459
(459, 51)
Epoch 1/30
459/459 [==============================] - 0s 782us/step - loss: 0.4924 - acc: 0.5076
Epoch 2/30
459/459 [==============================] - 0s 418us/step - loss: 0.5008 - acc: 0.4989
Epoch 3/30
459/459 [==============================] - 0s 631us/step - loss: 0.4924 - acc: 0.5076
Epoch 4/30
459/459 [==============================] - 0s 448us/step - loss: 0.4924 - acc: 0.5076
Epoch 5/30
459/459 [==============================] - 0s 337us/step - loss: 0.4924 - acc: 0.5076
Epoch 6/30
459/459 [==============================] - 0s 550us/step - loss: 0.4924 - acc: 0.5076
Epoch 7/30
459/459 [==============================] - 0s 457us/step - loss: 0.4924 - acc: 0.5076
Epoch 8/30
459/459 [==============================] - 0s 458us/step - loss: 0.4924 - acc: 0.5076
Epoch 9/30
459/459 [==============================] - 0s 332us/step - loss: 0.4924 - acc: 0.5076
Epoch 10/30
459/459 [==============================] - 0s 336us/step - loss: 0.4924 - acc: 0.5076
Epoch 

In [ ]:
# for ind, grp in play_win.groupby('POS'):
#     print('==================================')
#     print(ind)
#     reduced_play_win = grp[['2YR', 'ASSIST', 'CA', 'CC', \
#                             'DAT', 'DC', 'DCIP', 'DPA', \
#                             'DZONE1', 'DZONE2', 'DZONE3', 'DZONE4', 'DZONE5',\
#                             'FOUL', 'Long_PA', 'Long_PC', 'Medium_PA', 'Medium_PC',\
#                             'OFF', 'RC', 'SAVE', 'SCORE', 'SHOT', 'SPA', 'SPRINT', 'SRAT',\
#                             'SRPA', 'Short_PA', 'Short_PC', 'TA3RD', 'TG', 'TNG', 'TOH', \
#                             'TPA', 'TS', 'TSCORE', 'TSCORE_OG', 'TSCORE_PEN', 'TSL', 'TSNL', 'TTP',\
#                             'TZONE1', 'TZONE2', 'TZONE3', 'TZONE4', 'TZONE5', 'Total_PA',\
#                             'Total_PC', 'Total_PP', 'YC', 'winning_flag']]
    
#     print(len(reduced_play_win))
#     print(reduced_play_win.shape)
#     reduced_play_win.describe()
#     reduced_play_win.reset_index(drop=True)
#     reduced_play_win = reduced_play_win.values
#     X = reduced_play_win[:, :50]
#     Y = reduced_play_win[:, 50]
    
#     X_train, X_test, y_train, y_test = train_test_split(X, Y,  
#                                                     train_size=0.75,  
#                                                     test_size=0.25)  
    
#     tpot = TPOTClassifier()
#     tpot = TPOTClassifier(generations=10)  
#     tpot.fit(X_train, y_train)  

#     print(tpot.score(X_test, y_test)) 